<a href="https://colab.research.google.com/github/basmaeldrandaly/EC-algorithms/blob/main/Copy_of_AntColonyOptimization_VS_Genetic_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GA: 
      Time-> 477.1612322330475s
      Best-sol-> 573.6317665251336
# ACO:
      Time-> 1.424128770828247s
      Best-sol-> 539.9073278877743

 **It appears that in this problem ACO Clearly performs better than GA in terms of both speed and cost of selected solution**

# GA

In [15]:
# Imports 
import numpy as np
import random
import time

from datetime import datetime
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Amsterdam': [27, 47],
 'Barcelona': [83, 2],
 'Berlin': [49, 55],
 'Birmingham': [76, 18],
 'Brussels': [82, 89],
 'Bucharest': [55, 21],
 'Budapest': [94, 36],
 'Istanbul': [19, 77],
 'Kyiv': [51, 22],
 'London': [66, 17],
 'Milan': [62, 83],
 'Minsk': [8, 87],
 'Moscow': [86, 23],
 'Munich': [57, 3],
 'Paris': [97, 53],
 'Prague': [25, 53],
 'Rome': [34, 8],
 'Sofia': [77, 33],
 'Vienna': [14, 96],
 'Warsaw': [1, 35]}

In [31]:
np.random.choice(list(range(n_cities)), n_cities, replace=False)

array([15,  8,  0,  7, 14, 10, 12,  9, 17,  1, 19, 11, 16, 18,  3,  2, 13,
        5,  4,  6])

In [41]:
# First step: Create the first population set
import time
start=time.time()
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Amsterdam', 'Vienna', 'Paris', ..., 'Berlin', 'London', 'Minsk'],
       ['London', 'Istanbul', 'Prague', ..., 'Bucharest', 'Munich',
        'Milan'],
       ['Berlin', 'Paris', 'Brussels', ..., 'London', 'Sofia',
        'Bucharest'],
       ...,
       ['Warsaw', 'Paris', 'Minsk', ..., 'London', 'Rome', 'Munich'],
       ['Kyiv', 'Prague', 'Amsterdam', ..., 'Barcelona', 'Sofia',
        'Vienna'],
       ['Moscow', 'Rome', 'Sofia', ..., 'Istanbul', 'Milan', 'Warsaw']],
      dtype='<U10')

In [42]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([1112.08863438,  888.6972137 ,  872.06841029,  888.10078665,
       1122.68289228, 1129.03839359,  932.86845417,  862.50368129,
       1097.78463782, 1067.86719434, 1124.07178147,  962.99273253,
       1152.88204762, 1090.20309527,  894.78106394, 1130.41982016,
        822.47818855,  966.13775494, 1014.66655447, 1029.52873465,
       1162.80535045, 1054.69166166,  992.47578763, 1094.69334867,
       1014.42290704, 1136.29499642,  944.40195629,  942.08088703,
        968.12410612,  943.33752664, 1154.75325893,  936.64495996,
       1123.32168883, 1144.98396527,  950.28405066, 1113.06215995,
        884.33085474, 1002.6162601 ,  929.63061975, 1064.78466348,
       1179.9675141 , 1044.24080167, 1002.32226068, 1172.68333655,
        965.09478648,  946.70040981, 1080.17516983, 1063.3560266 ,
       1203.65930304, 1020.16847804,  900.72131923, 1036.90058802,
       1115.64308815,  856.69887967, 1104.37861455, 1177.36000256,
       1070.74752219, 1011.02343727,  993.76318734, 1149.56795

In [44]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    #the 2 parents selected
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
#the second parent
progenitor_list[0][2]

array(['Paris', 'Warsaw', 'Sofia', 'London', 'Kyiv', 'Minsk', 'Milan',
       'Munich', 'Prague', 'Rome', 'Barcelona', 'Brussels', 'Istanbul',
       'Budapest', 'Berlin', 'Birmingham', 'Bucharest', 'Vienna',
       'Moscow', 'Amsterdam'], dtype='<U10')

In [45]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Rome', 'Paris', 'Budapest', 'Warsaw', 'Bucharest', 'Prague',
       'Brussels', 'Minsk', 'Birmingham', 'Milan', 'London', 'Kyiv',
       'Moscow', 'Istanbul', 'Barcelona', 'Sofia', 'Vienna', 'Berlin',
       'Amsterdam', 'Munich'], dtype='<U10')

In [46]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)
#swap 2 cities within the same sol/offspring
        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Rome', 'Paris', 'Budapest', 'Sofia', 'Moscow', 'Milan',
       'Brussels', 'Minsk', 'Vienna', 'Birmingham', 'London', 'Kyiv',
       'Prague', 'Istanbul', 'Barcelona', 'Warsaw', 'Bucharest', 'Berlin',
       'Amsterdam', 'Munich'], dtype='<U10')

In [50]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)
end=time.time()
print(f"time:{end-start}s")

0 820.47668526422 1031.549030719565 10/06/22 23:44
100 850.6912214012078 1026.3699116837038 10/06/22 23:44
200 811.4977246667771 1036.2856305984615 10/06/22 23:44
300 860.8764801890403 1056.0649024309505 10/06/22 23:44
400 799.2083188791041 1025.343013889824 10/06/22 23:44
500 813.1754229674019 1033.7807604442962 10/06/22 23:44
600 818.4978731915725 1041.559494124505 10/06/22 23:44
700 843.2841852112489 1043.4398424576007 10/06/22 23:44
800 773.0389491329746 1027.7047956621445 10/06/22 23:44
900 866.4399180487708 1043.860864264411 10/06/22 23:44
1000 813.9104473823969 1051.086106266137 10/06/22 23:44
1100 723.5221403617984 1037.7029548554747 10/06/22 23:44
1200 776.342035725288 1020.4985101600967 10/06/22 23:44
1300 784.3843771067418 1008.5014294054346 10/06/22 23:44
1400 745.2327850648004 1043.3560525060302 10/06/22 23:44
1500 684.0776761927757 1027.2814532449504 10/06/22 23:44
1600 789.1850803355402 1019.6886940528726 10/06/22 23:44
1700 792.214475847025 1031.383199160781 10/06/22 23

In [51]:
best_solution

[121,
 573.6317665251336,
 array([['Moscow', 'London', 'Barcelona', 'Milan', 'Brussels', 'Paris',
         'Budapest', 'Amsterdam', 'Prague', 'Warsaw', 'Vienna', 'Minsk',
         'Istanbul', 'Berlin', 'Rome', 'Kyiv', 'Bucharest', 'Sofia',
         'Birmingham', 'Munich']], dtype='<U10')]

# ACO

In [ ]:
for x in cities_dict:
    for y in cities_dict:
        print(',' , compute_city_distance_coordinates(cities_dict[x],cities_dict[y]) )
    print('------------')

problem initialization (5 different random sol corresponding to 5 cities)

In [23]:
import numpy as np
from numpy import inf

#given values for the problems
#distance matrix between the cities
d = np.array([[0.0,41.6293165929973,48.91829923454004,62.96824596572466,49.33558553417604,48.041648597857254,53.907327887774215,52.61178575186362,37.20215047547655,33.06055050963308,
34.52535300326414,52.009614495783374,52.0,48.84669896727925,30.870698080866262,24.08318915758459,35.608987629529715,45.803929962395145,47.38143096192854,23.40939982143925]
          ,[ 41.6293165929973
, 0.0
, 20.8806130178211
, 22.67156809750927
, 33.24154027718932
, 47.50789408087881
, 94.57801012920498
, 16.64331697709324
, 76.21679604916491
, 15.811388300841896
, 11.704699910719626
, 90.9065454189081
, 67.446274915669
, 33.83784863137726
, 66.12110101926616
, 54.56189146281496
, 19.4164878389476
, 10.04987562112089
, 73.7563556583431
, 49.20365840057018]
          ,[48.91829923454004
, 20.8806130178211
, 0.0
, 21.213203435596427
, 54.120236510939236
, 31.953090617340916
, 102.53292154230269
, 35.22782990761707
, 86.05230967266364
, 35.014282800023196
, 31.064449134018133
, 100.89598604503551
, 85.84287972802403
, 15.264337522473747
, 64.62197768561404
, 67.97793759742936
, 13.45362404707371
, 11.180339887498949
, 66.12110101926616
, 63.694583757176716]
          ,[62.96824596572466
, 22.67156809750927
, 21.213203435596427
, 0.0
, 49.36598018878993
, 52.88667128870941
, 116.60617479361889
, 26.019223662515376
, 98.59513172565875
, 37.73592452822641
, 33.83784863137726
, 113.35784048754634
, 88.3911760301898
, 35.73513677041127
, 83.67795408588812
, 77.23341245859851
, 31.575306807693888
, 17.46424919657298
, 87.00574693662483
, 71.84010022264724]
          ,[49.33558553417604
, 33.24154027718932
, 54.120236510939236
, 49.36598018878993
, 0.0
, 77.42092740338364
, 90.24411338142782
, 23.53720459187964
, 70.61161377563892
, 22.02271554554524
, 24.698178070456937
, 83.16850365372699
, 42.638011210655684
, 66.2117814289874
, 80.05623023850174
, 45.89117562233506
, 49.73932046178355
, 43.174066289845804
, 94.15412895885129
, 39.6232255123179],[48.041648597857254
, 47.50789408087881
, 31.953090617340916
, 52.88667128870941
, 77.42092740338364
, 0.0
, 93.47726996441435
, 64.03124237432849
, 81.60882305241266
, 55.47071299343465
, 52.80151512977634
, 95.2732911156112
, 97.67292357659824
, 17.26267650163207
, 46.09772228646444
, 72.0
, 28.284271247461902
, 40.8166632639171
, 39.0
, 70.25667228100119],[53.907327887774215
, 94.57801012920498
, 102.53292154230269
, 116.60617479361889
, 90.24411338142782
, 93.47726996441435
, 0.0
, 102.45974819410792
, 19.6468827043885
, 82.73451516749222
, 85.47514258543241
, 10.816653826391969
, 62.369864518050704
, 100.0
, 49.72926703662542
, 44.384682042344295
, 89.09545442950498
, 99.6393496566492
, 68.3593446428504
, 50.695167422546305],[52.61178575186362
, 16.64331697709324
, 35.22782990761707
, 26.019223662515376
, 23.53720459187964
, 64.03124237432849
, 102.45974819410792
, 0.0
, 83.18653737234169
, 19.924858845171276
, 18.110770276274835
, 97.24710792614863
, 64.4980619863884
, 49.57822102496216
, 80.15609770940699
, 59.36328831862332
, 36.05551275463989
, 24.20743687382041
, 89.56003573022959
, 53.25410782277739],[37.20215047547655
, 76.21679604916491
, 86.05230967266364
, 98.59513172565875
, 70.61161377563892
, 81.60882305241266
, 19.6468827043885
, 83.18653737234169
, 0.0
, 63.631753079732135
, 66.57326790837296
, 14.866068747318506
, 45.69463863518345
, 85.47514258543241
, 43.41658669218482
, 24.73863375370596
, 72.80109889280519
, 82.03657720797473
, 64.13267497929586
, 31.04834939252005],[33.06055050963308
, 15.811388300841896
, 35.014282800023196
, 37.73592452822641
, 22.02271554554524
, 55.47071299343465
, 82.73451516749222
, 19.924858845171276
, 63.631753079732135
, 0.0
, 4.123105625617661
, 77.93587107359485
, 51.66236541235796
, 45.221676218380054
, 61.98386886924694
, 40.45985664828782
, 28.231188426986208
, 25.317977802344327
, 73.824115301167
, 34.655446902326915],[34.52535300326414
, 11.704699910719626
, 31.064449134018133
, 33.83784863137726
, 24.698178070456937
, 52.80151512977634
, 85.47514258543241
, 18.110770276274835
, 66.57326790837296
, 4.123105625617661
, 0.0
, 81.02468759581859
, 55.78530272392541
, 41.78516483155236
, 62.39390995922599
, 43.86342439892262
, 25.059928172283335
, 21.213203435596427
, 73.16419889536138
, 38.2099463490856],[52.009614495783374
, 90.9065454189081
, 100.89598604503551
, 113.35784048754634
, 83.16850365372699
, 95.2732911156112
, 10.816653826391969
, 97.24710792614863
, 14.866068747318506
, 77.93587107359485
, 81.02468759581859
, 0.0
, 52.46903848937962
, 99.98499887483122
, 54.147945482723536
, 38.01315561749642
, 87.6184911990614
, 96.87620966986684
, 74.02702209328699
, 44.28317965096906],[52.0
, 67.446274915669
, 85.84287972802403
, 88.3911760301898
, 42.638011210655684
, 97.67292357659824
, 62.369864518050704
, 64.4980619863884
, 45.69463863518345
, 51.66236541235796
, 55.78530272392541
, 52.46903848937962
, 0.0
, 93.00537618869137
, 77.62087348130012
, 30.0
, 76.02631123499285
, 76.90253571892151
, 97.3498844375277
, 28.635642126552707],[48.84669896727925
, 33.83784863137726
, 15.264337522473747
, 35.73513677041127
, 66.2117814289874
, 17.26267650163207
, 100.0
, 49.57822102496216
, 85.47514258543241
, 45.221676218380054
, 41.78516483155236
, 99.98499887483122
, 93.00537618869137
, 0.0
, 56.859475903318
, 71.06335201775947
, 17.26267650163207
, 25.45584412271571
, 54.3415126767741
, 67.89698078707183],[30.870698080866262
, 66.12110101926616
, 64.62197768561404
, 83.67795408588812
, 80.05623023850174
, 46.09772228646444
, 49.72926703662542
, 80.15609770940699
, 43.41658669218482
, 61.98386886924694
, 62.39390995922599
, 54.147945482723536
, 77.62087348130012
, 56.859475903318
, 0.0
, 47.634021455258214
, 52.20153254455275
, 66.49060083951716
, 20.8806130178211
, 50.20956084253277],[24.08318915758459
, 54.56189146281496
, 67.97793759742936
, 77.23341245859851
, 45.89117562233506
, 72.0
, 44.384682042344295
, 59.36328831862332
, 24.73863375370596
, 40.45985664828782
, 43.86342439892262
, 38.01315561749642
, 30.0
, 71.06335201775947
, 47.634021455258214
, 0.0
, 55.71355310873648
, 61.85466837676846
, 67.4166151627327
, 6.324555320336759],[35.608987629529715
, 19.4164878389476
, 13.45362404707371
, 31.575306807693888
, 49.73932046178355
, 28.284271247461902
, 89.09545442950498
, 36.05551275463989
, 72.80109889280519
, 28.231188426986208
, 25.059928172283335
, 87.6184911990614
, 76.02631123499285
, 17.26267650163207
, 52.20153254455275
, 55.71355310873648
, 0.0
, 15.033296378372908
, 56.22277118748239
, 51.92301994298868],[45.803929962395145
, 10.04987562112089
, 11.180339887498949
, 17.46424919657298
, 43.174066289845804
, 40.8166632639171
, 99.6393496566492
, 24.20743687382041
, 82.03657720797473
, 25.317977802344327
, 21.213203435596427
, 96.87620966986684
, 76.90253571892151
, 25.45584412271571
, 66.49060083951716
, 61.85466837676846
, 15.033296378372908
, 0.0
, 71.25307010929424
, 56.938563381947034],[47.38143096192854
, 73.7563556583431
, 66.12110101926616
, 87.00574693662483
, 94.15412895885129
, 39.0
, 68.3593446428504
, 89.56003573022959
, 64.13267497929586
, 73.824115301167
, 73.16419889536138
, 74.02702209328699
, 97.3498844375277
, 54.3415126767741
, 20.8806130178211
, 67.4166151627327
, 56.22277118748239
, 71.25307010929424
, 0.0
, 69.20260110718382],[23.40939982143925
, 49.20365840057018
, 63.694583757176716
, 71.84010022264724
, 39.6232255123179
, 70.25667228100119
, 50.695167422546305
, 53.25410782277739
, 31.04834939252005
, 34.655446902326915
, 38.2099463490856
, 44.28317965096906
, 28.635642126552707
, 67.89698078707183
, 50.20956084253277
, 6.324555320336759
, 51.92301994298868
, 56.938563381947034
, 69.20260110718382
, 0.0]])


iteration = 100
n_ants = 20
n_citys = 20

pheromone =1 for all cities at the beggining

In [24]:
# intialization part

m = n_ants
n = n_citys
e = .5         #evaporation rate
alpha = 1     #pheromone factor
beta = 2       #visibility factor

In [25]:
#calculating the visibility of the next city visibility(i,j)=1/d(i,j)
#if we can't decide with the pheremone, visibility uses distance so that ants can decide which city to choose based on the closer one

visibility = 1/d
visibility[visibility == inf ] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


In [26]:
#intializing pheromne present at the paths to the cities

#all ants start at location one
pheromne = .1*np.ones((m,n))

#intializing the rute of the ants with size rute(n_ants,n_citys+1) 
#note adding 1 because we want to come back to the source city

rute = np.ones((m,n+1))


In [27]:
pheromne = .1*np.ones((20,20))

In [28]:
pheromne

array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0

In [52]:
start=time.time()
for ite in range(iteration):
    
    rute[:,0] = 1          #initial starting and ending positon of every ants '1' i.e city '1'
    
    for i in range(m):
        
        temp_visibility = np.array(visibility)         #creating a copy of visibility
        
        #traverse for 4 cities (check last step to choose the remaining city which wasn't chosen)
        for j in range(n-1):
            #print(rute)
            
            combine_feature = np.zeros(20)     #intializing combine_feature array to zero
            cum_prob = np.zeros(20)            #intializing cummulative probability array to zeros
            
            cur_loc = int(rute[i,j]-1)        #current city of the ant
            
            #making the visibility of already visited cities as 0 (to not visit it again)
            temp_visibility[:,cur_loc] = 0     #making visibility of the current city as zero
            
            p_feature = np.power(pheromne[cur_loc,:],beta)         #calculating pheromne feature 
            v_feature = np.power(temp_visibility[cur_loc,:],alpha)  #calculating visibility feature
            
            p_feature = p_feature[:,np.newaxis]                     #adding axis to make a size[5,1]
            v_feature = v_feature[:,np.newaxis]                     #adding axis to make a size[5,1]
            
            #element by element multiplication
            combine_feature = np.multiply(p_feature,v_feature)     #calculating the combine feature
                        
            total = np.sum(combine_feature)                        #sum of all the feature
            
            probs = combine_feature/total   #finding probability of element probs(i) = comine_feature(i)/total
            
            cum_prob = np.cumsum(probs)     #calculating cummulative sum
            #print(cum_prob)

            r = np.random.random_sample()   #randon no in [0,1)
            #print(r)

            #pick the first element(city) from comulative prob that's greater than r
            city = np.nonzero(cum_prob>r)[0][0]+1       #finding the next city having probability higher then random(r) 
            #print(city)
            
            #adding the chosen city to the current route
            rute[i,j+1] = city              #adding city to route 
           

           #choose the remaining 5th city (one not visited yet)
        left = list(set([i for i in range(1,n+1)])-set(rute[i,:-2]))[0]     #finding the last untraversed city to route
        
        rute[i,-2] = left                   #adding untraversed city to route
       
    rute_opt = np.array(rute)               #intializing optimal route
    
    #calculating soution cost (route distance)
    dist_cost = np.zeros((m,1))             #intializing total_distance_of_tour with zero 
    
    for i in range(m):
        
        s = 0
        for j in range(n-1):
            
            s = s + d[int(rute_opt[i,j])-1,int(rute_opt[i,j+1])-1]   #calcualting total tour distance
        
        dist_cost[i]=s                      #storing distance of tour for 'i'th ant at location 'i' 
       
    dist_min_loc = np.argmin(dist_cost)             #finding location of minimum of dist_cost
    dist_min_cost = dist_cost[dist_min_loc]         #finging min of dist_cost
    
    best_route = rute[dist_min_loc,:]               #intializing current traversed as best route

    #updating phermone
    pheromne = (1-e)*pheromne                       #evaporation of pheromne with (1-e)
    

    #phermone of best route is more
    for i in range(m):
        for j in range(n-1):
            dt = 1/dist_cost[i]
            pheromne[int(rute_opt[i,j])-1,int(rute_opt[i,j+1])-1] = pheromne[int(rute_opt[i,j])-1,int(rute_opt[i,j+1])-1] + dt   
            #updating the pheromne with delta_distance
            #delta_distance will be more with min_dist i.e adding more weight to that route  peromne

print('route of all the ants at the end :')
print(rute_opt)
print()
print('best path :',best_route)
print('cost of the best path',int(dist_min_cost[0]) + d[int(best_route[-2])-1,0])
end=time.time()
print(f"time:{end-start}s")

            
            
            

route of all the ants at the end :
[[ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
  12.  7.  1.]
 [ 1. 11. 10.  5.  8.  4. 18.  2. 15. 19.  6. 14. 17.  3. 16. 20. 13.  9.
 